In [1]:
import os

from datasets import Dataset, load_dataset, concatenate_datasets
from openai import OpenAI
from ragas import evaluate

openai_api_key = os.getenv("OPENAI_API_KEY")

### データセットの準備

In [2]:
jaquad_dataset = load_dataset("SkelterLabsInc/JaQuAD")

combined_data = concatenate_datasets(
    [jaquad_dataset["train"], jaquad_dataset["validation"]]
)

# 前処理：正解データの準備
combined_data_processed = []
for item in combined_data:
    answer_text = item["answers"]["text"][0] if item["answers"]["text"] else ""
    answer_start = (
        item["answers"]["answer_start"][0] if item["answers"]["answer_start"] else -1
    )
    combined_data_processed.append(
        {
            "question": item["question"],
            "context": item["context"],
            "answer_text": answer_text,
            "answer_start": answer_start,
        },
    )


# データが多いため10件に絞る
combined_data_processed = combined_data_processed[:10]
combined_data_processed[0]

[{'question': '戦後日本のストーリー漫画の第一人者で、医学博士の一面もある漫画家は誰?',
  'context': '手塚治虫(てづかおさむ、本名:手塚治(読み同じ)、1928年(昭和3年)11月3日-1989年(平成元年)2月9日)は、日本の漫画家、アニメーター、アニメ監督である。\n戦後日本においてストーリー漫画の第一人者として、漫画表現の開拓者的な存在として活躍した。\n\n兵庫県宝塚市出身(出生は大阪府豊能郡豊中町、現在の豊中市)同市名誉市民である。\n大阪帝国大学附属医学専門部を卒業。\n医師免許取得のち医学博士(奈良県立医科大学・1961年)。',
  'answer_text': '手塚治虫',
  'answer_start': 0},
 {'question': '手塚治虫の出身地はどこになりますか?',
  'context': '手塚治虫(てづかおさむ、本名:手塚治(読み同じ)、1928年(昭和3年)11月3日-1989年(平成元年)2月9日)は、日本の漫画家、アニメーター、アニメ監督である。\n戦後日本においてストーリー漫画の第一人者として、漫画表現の開拓者的な存在として活躍した。\n\n兵庫県宝塚市出身(出生は大阪府豊能郡豊中町、現在の豊中市)同市名誉市民である。\n大阪帝国大学附属医学専門部を卒業。\n医師免許取得のち医学博士(奈良県立医科大学・1961年)。',
  'answer_text': '兵庫県宝塚市',
  'answer_start': 130},
 {'question': '手塚治虫の漫画家としてのデビュー作は何かな?',
  'context': '大阪帝国大学附属医学専門部在学中の1946年1月1日に4コマ漫画『マアチャンの日記帳』(『少国民新聞』連載)で漫画家としてデビューした。\n1947年、酒井七馬原案の描き下ろし単行本『新寶島』がベストセラーとなり、大阪に赤本ブームを引き起こす。\n1950年より漫画雑誌に登場、『鉄腕アトム』『ジャングル大帝』『リボンの騎士』といったヒット作を次々と手がけた。\n\n1963年、自作をもとに日本初となる30分枠のテレビアニメシリーズ『鉄腕アトム』を制作、現代につながる日本のテレビアニメ制作に多大な影響を及ぼした。\n1970年代には『ブラック・ジャック』『三

In [3]:
def generate_answer(question: str, context: str) -> str:
    """APIを使ってLLMによる解答を得る関数

    Question (str): 質問内容
        context (str): 解答の根拠になるテキスト情報

    Returns:
        response.choices[0].message.content(str): contextを元に回答したLLMの回答内容

    """
    response = OpenAI().chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "あなたは優秀なアシスタントです。これから渡すコンテキストを元に問題に答えてください。答えられる文字数は少ないので単語のみなど簡潔な回答を心がけてください。",
            },
            {
                "role": "user",
                "content": f"Context: {context}. Answer the question: {question}",
            },
        ],
        max_tokens=15,
        temperature=0,
    )
    return response.choices[0].message.content

In [5]:
for item in combined_data_processed:
    item["rag_answer_text"] = generate_answer(item["question"], item["context"])


### RAGASの実装

In [6]:
def assemble_ragas_dataset(input_data: list) -> Dataset:
    """ragas用にデータを整える関数

        input_data (list):

    Returns:
        Dataset: ragasに入力するためのデータセット

    """
    question_list = [item["question"] for item in input_data]
    context_list = [[item["context"]] for item in input_data]
    answer_list = [item["rag_answer_text"] for item in input_data]
    ground_truth_list = [item["answer_text"] for item in input_data]

    return Dataset.from_dict(
        {
            "question": question_list,
            "contexts": context_list,
            "answer": answer_list,
            "ground_truth": ground_truth_list,
        },
    )


ragas_input_ds = assemble_ragas_dataset(combined_data_processed)
results = evaluate(ragas_input_ds)

results.to_pandas()

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.


,question,contexts,answer,ground_truth,answer_relevancy,context_precision,faithfulness,context_recall
0,戦後日本のストーリー漫画の第一人者で、医学博士の一面もある漫画家は誰?,[手塚治虫(てづかおさむ、本名:手塚治(読み同じ)、1928年(昭和3年)11月3日-198...,手塚治虫,手塚治虫,0.831302,1.0,NaN,1.0
1,手塚治虫の出身地はどこになりますか?,[手塚治虫(てづかおさむ、本名:手塚治(読み同じ)、1928年(昭和3年)11月3日-198...,兵庫県宝塚市,兵庫県宝塚市,0.825807,1.0,NaN,1.0
2,手塚治虫の漫画家としてのデビュー作は何かな?,[大阪帝国大学附属医学専門部在学中の1946年1月1日に4コマ漫画『マアチャンの日記帳』(『...,『マアチャンの日記帳』,『マアチャンの日記帳』,0.825326,1.0,NaN,1.0
3,1947年に大阪に赤本ブームが起こったのはなぜ?,[大阪帝国大学附属医学専門部在学中の1946年1月1日に4コマ漫画『マアチャンの日記帳』(『...,『新寶島』のベストセラーによる。,『新寶島』がベストセラーとなり,0.000000,1.0,NaN,1.0
4,日本で一番初めに30分枠のテレビアニメとなった作品名は何?,[大阪帝国大学附属医学専門部在学中の1946年1月1日に4コマ漫画『マアチャンの日記帳』(『...,『鉄腕アトム』,『鉄腕アトム』,0.730350,1.0,0.0,1.0
5,手塚治虫の代表作である『鉄腕アトム』、『リボンの騎士』、『アドルフに告ぐ』の中で、最も遅く発...,[大阪帝国大学附属医学専門部在学中の1946年1月1日に4コマ漫画『マアチャンの日記帳』(『...,『アドルフに告ぐ』,『アドルフに告ぐ』,0.000000,1.0,0.0,1.0
6,手塚治虫が軍事色が強かった時期にも関わらず、精力的に漫画活動を行った学校の名前は?,[1941年、大阪府立北野中学校(現在の大阪府立北野高等学校)に入学した。\n時節柄軍事色が...,大阪府立北野中学校,大阪府立北野中学校,0.778617,1.0,NaN,1.0
7,『紙の砦』や『どついたれ』などの自伝的作品の中にも描写されている手塚治虫にとって命を落として...,[1941年、大阪府立北野中学校(現在の大阪府立北野高等学校)に入学した。\n時節柄軍事色が...,大阪大空襲,大阪大空襲,0.728643,1.0,NaN,0.0
8,『紙の砦』と『どついたれ』の作品はどちらが早く描かれたの?,[1941年、大阪府立北野中学校(現在の大阪府立北野高等学校)に入学した。\n時節柄軍事色が...,『紙の砦』,『紙の砦』,0.775315,1.0,NaN,0.0
9,『マアチャンの日記帳』の連載期間はいつからいつまで?,[終戦後、学生である手塚は戦時中に描き溜めた長編の中から『幽霊男』(『メトロポリス』の原型)...,1946年1月1日-3月31日,1946年1月1日-3月31日,0.772473,1.0,NaN,1.0
